<a href="https://colab.research.google.com/github/jnshsrs/image_augmentation/blob/main/running_dfu_image_augmentation_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Load Packages

Installation is required each time this notenook runs.

Storing the image data, the Google Drive is mounted for this notebook.

Then, packages are imported.



In [3]:
!pip install -U git+https://github.com/albumentations-team/albumentations

  Cloning https://github.com/albumentations-team/albumentations to /tmp/pip-req-build-gz3hdk9t
  Running command git clone -q https://github.com/albumentations-team/albumentations /tmp/pip-req-build-gz3hdk9t
  Created wheel for albumentations: filename=albumentations-0.5.2-cp37-none-any.whl size=97039 sha256=f37953a5d840ef88a16fefd0bf5e46b6c8bb4493c66bea0c55cc357e5f6c314e
  Stored in directory: /tmp/pip-ephem-wheel-cache-c61epauz/wheels/6f/77/82/86baf8aeda64a6de0f890cd0f2fb31acaf5545cc9c99ad21ba
Successfully built albumentations
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [4]:
from google.colab import drive
drive.mount('/content/diabetic-foot-ulcer')

Mounted at /content/diabetic-foot-ulcer


In [5]:
%matplotlib inline

In [6]:
import random

import cv2
from matplotlib import pyplot as plt

import albumentations as A

import os

from google.colab.patches import cv2_imshow

# Define Functions

## Display of Bounding Boxes

In [7]:
# Preparation

## Define the display of Bounding Boxes
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White

def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    
    x_min, y_min, x_max, y_max = bbox
    x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)

    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)

    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

## Get file names of PASCAL VOC Files

## Import of Bounding Boxes

In [8]:
import xml.etree.ElementTree as ET

def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    # filename = root.find('filename').text
    filename = xml_file.replace(".xml", ".jpg")
    list_with_all_boxes = []

    for boxes in root.iter('object'):

        ymin, xmin, ymax, xmax = None, None, None, None

        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text)

        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        list_with_all_boxes.append(list_with_single_boxes)

    return filename, list_with_all_boxes

In this project uses the *pascal yolo* format.

[Yolo Format](https://albumentations.ai/docs/images/getting_started/augmenting_bboxes/bbox_example.jpg)

## Write Transformed Pascal VOC bboxes

In [9]:
!pip install pascal-voc-writer

# Writer(path, width, height)
from pascal_voc_writer import Writer

def write_augmented_pascalvoc_bboxes(bboxes, fpath_and_fname_of_augmented_img, transformed_image):

  writer = Writer(fpath_and_fname_of_augmented_img, transformed_image.shape[0], transformed_image.shape[1])

  for i in range(len(bboxes)):
    xmin, ymin, xmax, ymax = bboxes[i]
    xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
    label = transformed_labels[i]
    writer.addObject(label, xmin, ymin, xmax, ymax)

  xml_fpath_and_fname = fpath_and_fname_of_augmented_img.replace(".jpg", ".xml")

  writer.save(xml_fpath_and_fname)
  print("BBOXES saved: " + xml_fpath_and_fname)

# Import Raw Data (bboxes and images)

First, the pascal voc, which is a xml file, is read.
Then, the jpg-image, which has the same file name as the xml file, is loaded and visualized.

In [ ]:
# specify the location of the images (should contain the bbox files as well)
fpath = "/content/diabetic-foot-ulcer/MyDrive/images-annotated/"


files = []
for file in os.listdir(fpath):
    if file.endswith(".xml"):
        files.append(os.path.join(fpath, file))

pascal_voc_bbox = files[1]

img_name, bboxes = read_content(pascal_voc_bbox)

image = cv2.imread(img_name)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

dim = (800, 800)
category_ids = [1]
category_id_to_name = {1: 'dfu'}

# raw image with bounding boxes
visualize(image, bboxes, category_ids, category_id_to_name)

# Simple resize transformation procedure

In [ ]:
# resize image
image_ = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
image_ = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cv2_imshow(image_)

# Define Transformation Pipeline

1. Transformation with multiple steps
2. Resize of the Raw Image without further augmentations

In [12]:
trans = [
    A.Resize(800, 800),
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=.4), 
    A.HueSaturationValue(p=.1, hue_shift_limit=5, sat_shift_limit=5, val_shift_limit=5),
    A.GaussianBlur(blur_limit=3, p=.2),
    A.MedianBlur(blur_limit=3, p=.2),
    A.RandomBrightnessContrast(brightness_limit=.2, contrast_limit=.2, p=1),
    A.ShiftScaleRotate(p=1),
    A.Rotate(p=.8),
    A.HorizontalFlip(p=.7)]

transform = A.Compose(
    trans,
    bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.3, label_fields=['class_labels'])
)


/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1852: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


In [13]:
trans_resize = [A.Resize(800, 800)]
transform_resize = A.Compose(trans_resize, bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.3, label_fields=['class_labels']))

## Define Metadata of the Image

1. Category ID
2. Category Label (dict)
3. Dimensions of resized images

In [14]:
dim = (800, 800)
class_labels = [1]
category_id_to_name = {1: 'dfu'}

# Bulk Import of Images

In [15]:
fpath = "/content/diabetic-foot-ulcer/MyDrive/images-annotated/"


files = []
for file in os.listdir(fpath):
    if file.endswith(".xml"):
        files.append(os.path.join(fpath, file))


In [ ]:

# Define File Path of Images with Annotations
fpath = "/content/diabetic-foot-ulcer/MyDrive/images-annotated/"
folder_for_augmented_images = "/content/diabetic-foot-ulcer/MyDrive/images-augmentated/"

# Get all Pascal VOC XML Files

#pascal_files = get_pascal_voc(fpath)
#print(pascal_files)

for f in files[1]:
  
  raw_img_name, raw_img_bboxes = read_content(f)
  
  if len(raw_img_bboxes) == 1:
    print("I start augmenting!")
    try:
      f = open(raw_img_name)
      # Do something with the file
    except IOError:
        print("File not accessible")
    finally:
        f.close()

    for i in range(10):
      image_ = cv2.imread(raw_img_name)
      image_ = cv2.cvtColor(image_, cv2.COLOR_BGR2RGB)

      # Augment labels
      transformed = transform(image = image_, bboxes=raw_img_bboxes, class_labels=class_labels)

      # Get objects from transformed label
      transformed_image = transformed['image']
      transformed_bboxes = transformed['bboxes']
      transformed_labels = transformed['class_labels']

      # cv2_imshow(transformed_image)
      # new image name
      fname = '_{:02}'.format(i) + ".jpg"
      file_path_for_augmented_image = raw_img_name.replace('.jpg', fname)
      fname_augmented_image = file_path_for_augmented_image.split("/")[-1]
      

      path_to_store_augmented_image_in_loop = folder_for_augmented_images + fname_augmented_image

      #cv2_imshow(transformed_image)
      # Write bboxes for augmented images
      write_augmented_pascalvoc_bboxes(bboxes = transformed_bboxes,
                                      fpath_and_fname_of_augmented_img = path_to_store_augmented_image_in_loop,
                                      transformed_image = transformed_image)
      
      # Save transformed image
      transformed_image_rgb = cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB)
      # cv2_imshow(transformed_image_rgb)
      cv2.imwrite(path_to_store_augmented_image_in_loop, transformed_image_rgb)

  else:
    print("Skipped; Number of BBoxes > 1")

In [ ]:
files[:, -1]